# Load data & check first 5 datas

In [15]:
import pandas as pd
import numpy as np
pf = pd.read_csv("/Users/hayden/Downloads/Projects/used_car_project/data/raw/used_cars.csv")
pf.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


# Check for duplicates, null values & column types

In [16]:
pf.info()
pf.describe()
pf.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


brand             0
model             0
model_year        0
milage            0
fuel_type       170
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

In [17]:
# Remove duplicates

pf = pf.drop_duplicates()

# Missing values

In [18]:
pf['accident'] = pf['accident'].fillna("No") # cars with no accidents had blanks
pf['fuel_type'] = pf['fuel_type'].fillna("Electric") # values with no fuel type were all electric cars
pf['clean_title'] = pf['clean_title'].fillna("No") # cars with no clean title had blanks


# Outliers

In [19]:
pf['price'] = pf['price'].replace('[$,]', '', regex=True).astype(float) # remove $ sign so price column could be a numeric column
pf = pf[pf['price'] < 300000]  # Assuming not many used cars are priced above 300k

# Remove unnecessary columns

In [20]:
pf = pf.drop(['transmission'], axis = 1)

# Mileage & bins

In [21]:
# mileage typo
pf.rename(columns={'milage' : 'mileage'}, inplace=True)

# change mileage into bare numbers
pf['mileage'] = pf['mileage'].astype(str)
pf['mileage'] = pf['mileage'].str.replace('mi.', '', case=False)
pf['mileage'] = pf['mileage'].str.replace(',', '')
pf['mileage'] = pf['mileage'].str.strip()
pf['mileage'] = pd.to_numeric(pf['mileage'], errors='coerce')

# sort mileage into bins
bins = [0, 20_000, 40_000, 60_000, 80_000, 100_000, 150_000, 200_000, np.inf]
labels = ["0-20k", "20k-40k", "40k-60k", "60k-80k", "80k-100k", "100k-150k", "150k-200k", "200k+"]
pf['mile_bin'] = pd.cut(pf['mileage'], bins=bins, labels=labels, right = True, include_lowest = True)


# Binary values
    Change yes/no into binary values

In [22]:
pf['accident'] = pf['accident'].map({'At least 1 accident or damage reported': 1, 'None reported': 0}) # 1 : accident, 0 : no accidents
pf['clean_title'] = pf['clean_title'].map({'Yes': 1, 'No': 0}) # 1 : clean title, 0 : not clean title
pf = pf.dropna(subset = ['accident'])

# Interrior/exterior cleaning
    replace '-' to 'other'

In [23]:
pf['int_col'] = pf['int_col'].replace('–', pd.NA)
pf['ext_col'] = pf['ext_col'].replace('–', pd.NA)

pf['ext_col'] = pf['ext_col'].fillna("other")
pf['int_col'] = pf['int_col'].fillna("other")


# Age of the car

In [24]:
Year = 2025
pf['car_age'] = Year - pf['model_year']
pf = pf.drop(['model_year'], axis=1)

# Price per mile

In [25]:
pf['price_per_mile'] = pf['price']/pf['mileage'].replace({0: np.nan})

# Group ext color
    6 most popular colors and group other colors

In [26]:
pf['ext_col_grouped'] = np.where(pf['ext_col'].isin(['Black', 'Blue', 'Silver', 'White', 'Gray', 'Red']), pf['ext_col'], 'other')

# Export cleaned data into clean data file

In [27]:
pf.to_csv("/Users/hayden/Downloads/Projects/used_car_project/data/cleaned/used_cars_cleaned.csv", index=False)

In [28]:
pf.info()
pf.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 3868 entries, 0 to 4008
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   brand            3868 non-null   object  
 1   model            3868 non-null   object  
 2   mileage          3868 non-null   int64   
 3   fuel_type        3868 non-null   object  
 4   engine           3868 non-null   object  
 5   ext_col          3868 non-null   object  
 6   int_col          3868 non-null   object  
 7   accident         3868 non-null   float64 
 8   clean_title      3868 non-null   int64   
 9   price            3868 non-null   float64 
 10  mile_bin         3868 non-null   category
 11  car_age          3868 non-null   int64   
 12  price_per_mile   3868 non-null   float64 
 13  ext_col_grouped  3868 non-null   object  
dtypes: category(1), float64(3), int64(3), object(7)
memory usage: 427.2+ KB


brand              0
model              0
mileage            0
fuel_type          0
engine             0
ext_col            0
int_col            0
accident           0
clean_title        0
price              0
mile_bin           0
car_age            0
price_per_mile     0
ext_col_grouped    0
dtype: int64